In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2

In [2]:
df = pd.read_csv("/home/yash/Downloads/Titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.drop(columns = ["PassengerId","Name","Ticket","Cabin"],inplace = True)
# inplace = True -> it modify the original dataframe
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [4]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns = ["Survived"]),df["Survived"],test_size = 0.1, random_state = 0)
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
815,1,male,NaN,0,0,0.0000,S
877,3,male,19.0,0,0,7.8958,S
193,2,male,3.0,1,1,26.0000,S
523,1,female,44.0,0,1,57.9792,C
634,3,female,9.0,3,2,27.9000,S


In [5]:
# impute transformer
# Age, Embarked
trf1 = ColumnTransformer(transformers = [
    ("impute_age",SimpleImputer(),[2]),
    ("impute_Embarked",SimpleImputer(strategy = "most_frequent"),[6])
    ],
    remainder = "passthrough"
)
# [2] = ["age"] index

In [6]:
# one hot encoding
# Sex, Embarked
trf2 = ColumnTransformer(transformers = [ 
    ("one_hot_encoding_Sex_Embarked",OneHotEncoder(sparse_output = False,handle_unknown="ignore"),[1,6])
    ],
    remainder = "passthrough")

In [7]:
# scaling
trf3 = ColumnTransformer(transformers = [
    ("scale",MinMaxScaler(),slice(0,10))
    ])

In [8]:
# feature selection
trf4 = SelectKBest(score_func = chi2, k = 8)
# On top 4 column 

In [9]:
# train hte model
trf5 = DecisionTreeClassifier()

## Creat Pipeline

In [10]:
pipe = Pipeline([
    ("trf1", trf1),
    ("trf2", trf2),
    ("trf3", trf3),
    ("trf4", trf4),
    ("trf5", trf5)
])

## Pipeline Vs make_pipeline
#### Pipeline requires naming of steps but make_pipeline doesn't require anything
#### ( same applies to CollumnTransformer vs make_column_transformer ) 

In [11]:
# Alternate Syntax
# pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5) 

In [12]:
# train the model
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_Embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot_encoding_Sex_Embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7fd5869d18a0>)),
                ('trf5', DecisionTreeClassifier())])

In [13]:
from sklearn import set_config
set_config(display = "diagram")

In [14]:
# Predict Output
y_pred = pipe.predict(x_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred)*100)
print(y_pred)

62.22222222222222
[1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0
 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1]


### Explore Pipeline

In [15]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_Embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('one_hot_encoding_Sex_Embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x7fd5869d18a0>),
 'trf5': DecisionTreeClassifier()}

In [16]:
pipe.named_steps["trf1"].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_Embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder',
  FunctionTransformer(accept_sparse=True, check_inverse=False,
                      feature_names_out='one-to-one'),
  [0, 1, 3, 4, 5])]

In [17]:
pipe.named_steps["trf1"].transformers_[1]

('impute_Embarked', SimpleImputer(strategy='most_frequent'), [6])

In [18]:
pipe.named_steps["trf1"].transformers_[1][1]

SimpleImputer(strategy='most_frequent')

In [19]:
pipe.named_steps["trf1"].transformers_[1][1].statistics_

array(['S'], dtype=object)

### Cross Validation using Pipeline

In [20]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,x_train,y_train,cv = 5,scoring = "accuracy").mean()*100

np.float64(63.7942546583851)